In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Lambda, BatchNormalization,LSTM,TimeDistributed,Activation,Dense)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint   
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import os 

  
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import random

voice_data=[]
for dirname, _, filenames in os.walk('../input/voicerecognitiondata/asr_bengali/data'):
    for filename in filenames:
        voice_data.append(os.path.join(dirname, filename))
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tf.test.is_gpu_available()

In [ ]:
#..........fixed the seed................
def reset_random_seeds():
    seed_num=9
    os.environ['PYTHONHASHSEED']=str(seed_num)
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

In [ ]:
#read voice data
#print(len(voice_data))
tsv_data=pd.read_csv("../input/voicerecognitiondata/asr_bengali/utt_spk_text.tsv",sep="\t",header=0)
#from kaggle_datasets import KaggleDatasets
#Datset="/kaggle/input/voicerecognitiondata/asr_bengali/data"
#GCS_PATH = KaggleDatasets().get_gcs_path(Datset )

#train_filenames1 = tf.io.gfile.glob(GCS_PATH1 + '*/*.flac')
tsv_data.head()

In [ ]:
#add header in tsv file
#tsv_data_1=pd.DataFrame(tsv_data,columns = ["wav_name","id","label"])

#tsv_data_1.head()
tsv_data.columns=["wav_name","id","label"]
tsv_data_1=tsv_data

In [ ]:
tsv_data.shape[0]

In [ ]:
print(voice_data[0])
print(voice_data[0].split("/")[-1].replace(".flac",""))

In [ ]:
#import zipfile
#with zipfile.ZipFile("/kaggle/input/voicerecognitiondata/asr_bengali/data/",'r') as zip_ref:
    #zip_ref.extractall("/kaggle/input/voicerecognitiondata/")

In [ ]:
#convert voice corpuses from .flac to wav format

from pydub import AudioSegment

#save wav file this location
#wav_root="../input/voicerecognitiondata/asr_bengali/data/wav"

os.makedirs('../outputs')
for root, _, files in os.walk('../input/voicerecognitiondata/asr_bengali/data'):
    for file in files:
        wav_name = file.replace(".flac",".wav")
        #print(wav_name)
        #print(root)
        try:
            # convert wav to mp3 
            sound = AudioSegment.from_file("{}/{}".format(root,file))
            #print(sound)
            sound.export("{}/{}".format('../outputs', wav_name), format="wav")
        except Exception as e:
            pass

In [ ]:
#.........read data wav data..................
#for dirname, _, filenames in os.walk("../outputs"):
    #for filename in filenames:
        #voice_data.append(
        #print(os.path.join(dirname, filename))

In [ ]:
# ............Mel-frequency cepstral coefficients..........................
#and converting vector form that are readable by deep learning model

def wav2mfcc(file_path,max_len, n_mfcc):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    #print(wave)
    wave = np.asfortranarray(wave[::3])
    #print(wave)
    #print(sr)
    
    mfcc = librosa.feature.mfcc(wave,sr=sr,n_mfcc=n_mfcc) #sr means sampling rate=16000
    mfcc_1=mfcc
    #print(mfcc.shape[0],mfcc.shape[1])
    
    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc,wave,mfcc_1.shape[0],mfcc_1.shape[1]

In [ ]:
#ordering the voice corpuses
from tkinter import Tcl

def save_data_to_array(path, max_len, n_mfcc):
    #labels,_, _ = get_labels()

    #for label in labels:
        # Init mfcc vectors
   
    #read audio file
    audio=[]
    for wav in path:
        wavfile=f'../outputs/{wav}'
        #wavfile=Tcl().call('lsort', '-dict',wavfile)
        #print(wavfile)
        audio.append(wavfile)
    
    #.............sort audio file.......
    sort_audio=Tcl().call('lsort', '-dict',audio) 
    print(sort_audio[0])
    #print(audio)
    audio_path=[]
    frequency_m=[]
    amplitude_m=[]
    mfcc_vectors = []
    for path in sort_audio:
        mfcc,wave,fre_sh,amp_sh= wav2mfcc(path, max_len, n_mfcc)
        audio_path.append(path.split("/")[-1].replace(".wav",""))
        mfcc_vectors.append(mfcc)
        frequency_m.append(fre_sh)
        amplitude_m.append(amp_sh)
    
    print(max(frequency_m))    
    print(max(amplitude_m))
    #np.save('E:/speech_recognition/wav', mfcc_vectors)
    return mfcc_vectors,wave,audio,audio_path

In [ ]:
#read wav file path
root_dir=os.listdir("../outputs")
mfcc_vec,wave,audio,audio_path=save_data_to_array(root_dir,177,34)#34 means time frequency and 384 amplitude
#print(mfcc_vec[0].shape[0])

In [ ]:
data={"mfcc":mfcc_vec,
      "wav_name":audio_path}
df=pd.DataFrame(data,columns=['mfcc',"wav_name"])
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape[0]

In [ ]:
tsv_data_1.head()

In [ ]:
print(df.shape[0])
print(tsv_data_1.shape[0])

In [ ]:
#merge the on basis of wav_name
df_new=pd.merge(df,tsv_data_1,on="wav_name",how="outer")
print(df_new.shape[0])

In [ ]:
print(df_new.isnull().sum())

In [ ]:
#now remove those rows which have a Nan value.

df_new.dropna(subset = ["label","mfcc"], inplace=True)
df_new.head()

In [ ]:
print(df_new.isnull().sum())

In [ ]:
df_new.shape[0]

In [ ]:
#find a label by mapping
#new_label=[]
#for i in range(df.shape[0]):
    #for j in range(tsv_data_1.shape[0]):
        
        #if df["audio_path"][i]==tsv_data_1["wav_name"][j]:
             
            #new_label.append(tsv_data_1["label"][j])
            #break      
        #else:
            #continue

In [ ]:
#df["transcript_label"]=new_label

In [ ]:
#..........clean text labels like html tags,quotation etc.............
import re
def clean(text):
    
    #text=text.map(lambda i:re.sub(r'[\\|!|?|\'|"|#|,|।|-|.|)|(|\|/|""|\n|]',r' ',str(i)))
    #text=text.map(lambda i:re.sub(r"\s+[a-zA-Z]\s+", ' ', str(i)))
    text=re.sub(r'[\\|!|?|\'|"|#|,|।|-|.|)|(|\|/|""|\n|]',r' ',str(text))
    text=re.sub(r"\s+[a-zA-Z]\s+", ' ', str(text))
    text=re.sub(r"[\u200d|’|\x93|\x93|\u200c|v|b|s|]", '', str(text))
    
    text=text.replace("।","")
    text=text.replace('-','')
    text=text.replace(':','')
    text=text.replace("\x94","")
    return text

df_new["label"]=df_new["label"].apply(clean)
#df_new["label"]=df_new["label"].apply(lambda i:re.sub(r"\s+[a-zA-Z]\s+", ' ', str(i)))
#df_new["label"]=clean(df_new["label"])
#print(clean(df_new["label"][32:35]))
df_new.head(3)

In [ ]:
def csv_file(df_new):
    return df_new

#csv_file(df_new)

In [ ]:
final_df=df_new

In [ ]:
final_df.to_csv('final_df.csv',index=False)

In [ ]:
#to reduce size the sample for memory issue
final_df=final_df[0:2000]
final_df.head()

In [ ]:
#...measure a label of label string.....................
final_df["label_len"]=[len(lab) for lab in final_df["label"]]
final_df.head()

In [ ]:
#....remove rows which have label length greater input length............

#i=final_df[len(final_df.label>120)].index

final_df.drop(final_df[final_df.label_len>30].index, inplace=True)
final_df.head()

In [ ]:
print(final_df.shape)

In [ ]:
#convert bangla character to number value
char_map_str = """
ঀ 0
ঁ 1
ং 2
ঃ 3
অ 4
আ 5
ই 6
ঈ 7
উ 8
ঊ 9
ঋ 10
ঌ 11
এ 12
ঐ 13
ও 14
ঔ 15
ক 16
খ 17
গ 18
ঘ 19
ঙ 20
চ 21
ছ 22
জ 23
ঝ 24
ঞ 25
ট 26
ঠ 27
ড 28
ঢ 29
ণ 30
ত 31
থ 32
দ 33
ধ 34
ন 35
প 36
ফ 37
ব 38
ভ 39
ম 40
য 41
র 42
ল 43
শ 44
ষ 45
স 46
হ 47
় 48
ঽ 49
া 50
ি 51
ী 52
ু 53
ূ 54
ৃ 55
ৄ 56
ে 57
ৈ 58
ো 59
ৌ 60
্ 61
ৎ 62
ৗ 63
ড় 64
ঢ় 65
য় 66
ৠ 67
০ 68
১ 69
২ 70
৩ 71
৪ 72
৫ 73
৬ 74
৭ 75
৮ 76
৯ 77
ৱ 78
৲ 79
৴ 80
<SPACE> 81
"""
# the "blank" character is mapped to 81
char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)] = ch
index_map[81] = ' '
 

In [ ]:
print(char_map)
print("****************************************************")
print(index_map)

In [ ]:
def get_number_of_char_classes(char_map):
    ## TODO would be better to check with dataset (once cleaned)
    num_classes = len(char_map)+1 #need +1 for ctc null char +1 pad
    return num_classes
get_number_of_char_classes(char_map)

In [ ]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        else:
            ch = char_map[c]
        int_sequence.append(ch)
    return int_sequence

def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    text = []
    for c in int_sequence:
        ch = index_map[c]
        text.append(ch)
    return text

In [ ]:
#...........normalize the feature...............
def calc_feat_dim(window, max_freq):
    return int(0.001 * window * max_freq) + 1

def normalize_feature(feature, eps=1e-14):
         
        #feat_dim = calc_feat_dim(34,8000)
        #feats_mean = np.zeros((feat_dim,))
        #feats_std = np.ones((feat_dim,))
        
        feats = np.vstack(feature)
        feats_mean = np.mean(feats, axis=0)
        feats_std = np.std(feats, axis=0)
        
        return (feature - feats_mean) / (feats_std + eps)

In [ ]:
#.........data preparation for model........
from past.builtins import xrange

def map_input_data(final_df):
    
    max_length = max([x.shape[0] for x in final_df['mfcc']])
    max_string_length = max([len(x) for x in final_df['label']])
    #print(max_string_length)
    X_data = np.zeros([final_df.shape[0], max_length,177])#here 177 means input shape
    #print(X_data)
    labels = np.ones([final_df.shape[0], max_string_length])*81
    input_length = np.zeros([final_df.shape[0], 1])
    label_length = np.zeros([final_df.shape[0], 1])
    #print(len(label_length))
    for i in range(0, final_df.shape[0]):
        feat = final_df.iloc[i]['mfcc']
        input_length[i] = feat.shape[0]
        #print(input_length)
        X_data[i, :feat.shape[0], :] = feat
        #print(X_data[i, :feat.shape[0], :])
        #print(len(final_df.iloc[i]['utterance']))
        # calculate labels & label_length
        #label = np.array(final_df.iloc[i]['utterance'])
        y=final_df.iloc[i]['label']
        #y=[' '.join(y[i:i+19]) for i in xrange(0,len(y),19)]                                                  
        #print(y)
        label=np.array(text_to_int_sequence(y))
        #print((label))
        #print(len(label))
        labels[i, :len(label)] = label
        #if feat.shape[0]<=len(label):
            #print(feat.shape[0])
            #label_length[i]=[' '.join(str(label[i:i+19])) for i in range(0,len(label),19)] 
        label_length[i] = len(label)

    #print(X_data)
    X_data=normalize_feature(X_data,eps=1e-14)
    # return the arrays
    outputs = {'ctc': np.zeros([final_df.shape[0]])}
    #outputs = {'ctc': np.zeros(20)}
     
    print(max(label_length))   
    inputs = {'input': X_data,   
              'labels': labels,
              'input_length':input_length ,
              'label_length':label_length
              }
     
    
    #return (inputs,outputs),(inputs_1,outputs_1),X_data,labels,input_length,label_length
    return (inputs,outputs)

In [ ]:
(input_dict,outputs)=map_input_data(final_df)

In [ ]:
#print(input_dict)

In [ ]:
#x=pd.DataFrame.from_dict(input_dict)
#x.head()

In [ ]:
#define ctc_lambda and ctc cost function
def ctc_lambda_func(args):
    #import tensorflow as tf
    
    y_pred, labels, input_length, label_length = args
    #y_pred = y_pred[:, 2:, :]
    #print(y_pred.shape)
    #print(input_length)
    #print(labels.shape)
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
    #return tf.compat.v1.nn.ctc_loss(labels, y_pred, input_length,label_length,
                                    #ignore_longer_outputs_than_inputs=True)

def add_ctc_loss(input_to_softmax):
    the_labels = Input(name='labels', shape=(None,), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='int64')
    label_lengths = Input(name='label_length', shape=(1,), dtype='int64')
    output_lengths = Lambda(input_to_softmax.output_length)(input_lengths)
    print(output_lengths)
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
    [input_to_softmax.output, the_labels, output_lengths, label_lengths])
    model = Model(
        inputs=[input_to_softmax.input, the_labels, input_lengths,label_lengths],
        outputs=loss_out)
    return model


In [ ]:
def cnn_output_length(input_length, filter_size, border_mode, stride,
                       dilation=1):
    """ Compute the length of the output sequence after 1D convolution along
        time. Note that this function is in line with the function used in
        Convolution1D class from Keras.
    Params:
        input_length (int): Length of the input sequence.
        filter_size (int): Width of the convolution kernel.
        border_mode (str): Only support `same` or `valid`.
        stride (int): Stride size used in 1D convolution.
        dilation (int)
    """
    if input_length is None:
        return None
    assert border_mode in {'same', 'valid'}
    dilated_filter_size = filter_size + (filter_size - 1) * (dilation - 1)
    if border_mode == 'same':
        output_length = input_length
    elif border_mode == 'valid':
        output_length = input_length - dilated_filter_size + 1
        #print(output_length )
    return (output_length + stride - 1) // stride
    #return output_length

In [ ]:
#.............................model-2...........................................
#...........................bi-directional deep rnn.........

from keras.layers import Dense, Activation, Bidirectional, Reshape, Flatten, Lambda, Input,\
    Masking, Convolution1D, BatchNormalization, GRU, Conv1D, RepeatVector, Conv2D,Dropout
from keras.optimizers import SGD, Adam

def final_model_1(input_dim, filters,kernel_size,conv_stride,
                  conv_border_mode,output_dim,number_of_layers,units):
    """ Build a deep network for speech 
    """
    dropout_rate=0.5
    cell=GRU 
    activation='tanh'
    
    # Main acoustic input
    input_data = Input(name='input', shape=(None, input_dim))
    # TODO: Specify the layers in your network
    conv_1d = Conv1D(filters, kernel_size,
                     strides=conv_stride,
                     padding=conv_border_mode,
                     activation='relu',
                     name='layer_1_conv',
                     dilation_rate=1)(input_data)
    conv_bn = BatchNormalization(name='conv_batch_norm')(conv_1d)
    
    conv_bn=Dropout(0.25)(conv_bn)
    if number_of_layers == 1:
        layer = cell(units, activation=activation,
                     return_sequences=True, implementation=2, name='rnn_1', dropout=dropout_rate)(conv_bn)
        layer = BatchNormalization(name='bt_rnn_1')(layer)
        layer=Dropout(0.25)(layer)
    else:
        layer = cell(units, activation=activation,
                     return_sequences=True, implementation=2, name='rnn_1', dropout=dropout_rate)(conv_bn)
        
        layer = BatchNormalization(name='bt_rnn_1')(layer)
        layer=Dropout(0.25)(layer)

        for i in range(number_of_layers - 2):
            layer = cell(units, activation=activation,
                         return_sequences=True, implementation=2, name='rnn_{}'.format(i + 2), dropout=dropout_rate)(layer)
            
            layer = BatchNormalization(name='bt_rnn_{}'.format(i + 2))(layer)

        layer = cell(units, activation=activation,
                     return_sequences=True, implementation=2, name='final_layer_of_rnn')(layer)
        layer=Dropout(0.25)(layer)
        layer = BatchNormalization(name='bt_rnn_final')(layer)

    time_dense = TimeDistributed(Dense(output_dim))(layer)
    time_dense=Dropout(0.5)(time_dense)
    # TODO: Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    # TODO: Specify model.output_length
    #model=add(Dropout(0.5))
    model.output_length = lambda x:x
          #cnn_output_length(x,5,conv_border_mode,2)
         #cnn_output_length(x, kernel_size,conv_border_mode,conv_stride)
    
    print(model.summary())
    return model

In [ ]:
#.................run the model.......................

def train_model(X,Y,optimizer=SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),epochs=20,verbose=1):
    import tensorflow as tf
    import math
    
    reset_random_seeds()
    #model_1 = deep_rnn_model(input_dim=384)
    model_1=final_model_1(input_dim=177,filters=200,kernel_size=1,conv_stride=1,
                                                    conv_border_mode='valid',
                                                    output_dim=83,
                                                    number_of_layers=3,
                                                    units=200)
    model=model_1
    model = add_ctc_loss(model)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    #model.compile(loss=ctc, optimizer=optimizer)
    #model.compile(loss=tf.compat.v1.nn.ctc_loss(the_labels=ctc,
                            #ignore_longer_outputs_than_inputs=True), optimizer=optimizer)
                                                    
    #if not os.path.exists('results'):
        #os.makedirs('results')
        #checkpointer = ModelCheckpoint(filepath='results/ASR_model.h5', verbose=0)
    #hist = model.fit(x=X,y=Y,batch_size=10,epochs=epochs,validation_split=0.25)
    #x=(X,Y)
    #x_1=(X_1,Y_1)
    #n_points =3
    #batch_size =10
    train_steps = math.ceil(3/10)
    valid_steps=math.ceil(1/10)
    
    #os.makedirs('../outputs1/cp.ckpt')
    
    #checkpoint_path = "E:/ssl_speech_recognition/cp.ckpt"
    checkpoint_dir = os.path.dirname("../outputs1/cp.ckpt")

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)

    #hist = model.fit_generator(generator=next_train(x,10),
                              #steps_per_epoch=train_steps,
                              #epochs=20,
                              #validation_data=next_valid(x_1,10),
                               #validation_steps=valid_steps,
                               #callbacks=[cp_callback])
                                    
    hist = model.fit(x=X,y=Y,batch_size=10,epochs=500,validation_split=0.35,
                    callbacks=[cp_callback])   #epoches=300,50000                        
    return (hist,model,model_1)                     

In [ ]:
hist,model,model_1=train_model(input_dict,outputs)

In [ ]:
#plot the loss 
history_dict = hist.history
keys = history_dict.keys()
loss = history_dict['loss']
val_loss = history_dict['val_loss']
 
plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss,  label='Training Loss')
plt.plot(epochs, val_loss,  label='Validation Loss')
plt.title('Loss vs Epochs', fontsize = 25)
plt.xlabel('Epochs', fontsize = 15)
plt.ylabel('Loss', fontsize = 15)
plt.legend()
plt.grid(True)
plt.show()